In [1]:
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Таблица *Event*
    Хранит данные о событиях, которые совершают пользователи. По сути, каждое событие — это факт прохождения пользователем какого-либо этапа игры.

    Название поля	 Описание
    id	            идентификатор события
    user_id	       уникальный идентификатор пользователя, совершившего событие в приложении
    start_time	    дата и время события
    event_type	    тип события 
                (значения: registration — регистрация; 
                tutorial_start — начало обучения; 
                tutorial_finish — завершение обучения; 
                level_choice — выбор уровня сложности; 
                pack_choice — выбор пакетов вопросов)
    tutorial_id	   идентификатор обучения (этот идентификатор есть только у событий обучения)
    selected_level	выбранный уровень сложности обучения

In [3]:
events = pd.read_csv('Events.csv', sep=',')


In [4]:
events_2018 = (events.start_time>='2018-01-01') & (events.start_time<'2019-01-01') & (events.event_type=='registration')
registered = events[events_2018]['user_id'].to_list() # список пользователей, зарег. в 2018
events_df = events[events.user_id.isin(registered)]

In [5]:
events_df['start_time'] = pd.to_datetime(events_df['start_time'])

## Таблица *purchase*
    Хранит данные об оплатах, которые совершают пользователи.

    Название поля	 Описание
    id	            идентификатор события
    user_id	       уникальный идентификатор пользователя, совершившего событие в приложении
    event_datetime	дата и время события/покупки
    amount	        сумма оплаты

In [6]:
purchase = pd.read_csv('purchase.csv', sep=',')


In [7]:
purchase['event_datetime'] = pd.to_datetime(purchase['event_datetime'])

In [8]:
purchase_df = purchase[purchase.user_id.isin(registered)]

In [9]:
events_df = events_df.rename(columns={"id": "event_id"})
purchase_df = purchase_df.rename(columns={"id": "purchase_id"})

In [10]:
purchase_df['event_type'] = 'purchase' # Добавляем в датафрейм purchase_df столбец event_type, который будет содержать одно значение purchase.
                                       # Это нужно, чтобы в объединённом датафрейме однозначно выделить события оплаты.

In [11]:
total_events_df = pd.concat([events_df,purchase_df],sort=False) # объединение датафреймов

In [12]:
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time') # сброс индексов + сортировка событий по возрастанию времени

## ЗАДАНИЕ
Необходимо проверить:

- есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты;
- различается ли временной промежуток между выбором уровня сложности и оплатой у групп пользователей с разным уровнем сложности
- различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.

### Зависимость между выбранным уровнем сложности и вероятностью оплаты

#### Уровень сложности easy

In [13]:
selected_level_easy_df = total_events_df[total_events_df["selected_level"] == "easy"] # Создаем датафрейм, с выбором уровня "easy"

In [14]:
selected_level_easy_df["user_id"].value_counts().mean() # Проверка сколько таких событий приходится на пользователя

1.0

##### Ищем пользователей с уровнем сложности easy

In [15]:
selected_level_easy_users = selected_level_easy_df["user_id"].unique()
len(selected_level_easy_users) # кол-во пользователей с уровнем easy

2448

##### Оплата от пользователей с уровнем сложности easy

In [16]:
purchase_df_easy = purchase_df[purchase_df["user_id"].isin(selected_level_easy_users)] # датафрейм с оплатой

In [17]:
purchase_df_easy['user_id'].nunique() #кол-во пользователей с оплатой  и уровнем easy

189

In [18]:
percent_of_purchase_easy = purchase_df_easy["user_id"].nunique() / len(
    selected_level_easy_users
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности easy): {:.2%}".format(
        percent_of_purchase_easy
    )
)

Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности easy): 7.72%


In [19]:
round(purchase_df_easy['amount'].mean(), 2) # средний размер оплаты

114.95

#### Уровень сложности medium

In [20]:
selected_level_medium_df = total_events_df[total_events_df["selected_level"] == "medium"] # Создаем датафрейм, с выбором уровня "medium"

In [21]:
selected_level_medium_df["user_id"].value_counts().mean() # Проверка сколько таких событий приходится на пользователя

1.0

##### Ищем пользователей с уровнем сложности medium

In [22]:
selected_level_medium_users = selected_level_medium_df["user_id"].unique()
len(selected_level_medium_users) # кол-во пользователей с уровнем medium

4645

##### Оплата от пользователей с уровнем сложности medium

In [23]:
purchase_df_medium = purchase_df[purchase_df["user_id"].isin(selected_level_medium_users)] # датафрейм с оплатой

In [24]:
purchase_df_medium['user_id'].nunique() #кол-во пользователей с оплатой  и уровнем medium

969

In [25]:
percent_of_purchase_medium = purchase_df_medium["user_id"].nunique() / len(
    selected_level_medium_users
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности medium): {:.2%}".format(
        percent_of_purchase_medium
    )
)

Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности medium): 20.86%


In [26]:
round(purchase_df_medium['amount'].mean(), 2) # средний размер оплаты

109.52

#### Уровень сложности hard

In [27]:
selected_level_hard_df = total_events_df[total_events_df["selected_level"] == "hard"] # Создаем датафрейм, с выбором уровня "hard"

In [28]:
selected_level_hard_df["user_id"].value_counts().mean() # Проверка сколько таких событий приходится на пользователя

1.0

##### Ищем пользователей с уровнем сложности hard

In [29]:
selected_level_hard_users = selected_level_hard_df["user_id"].unique()
len(selected_level_hard_users) # кол-во пользователей с уровнем hard

1249

##### Оплата от пользователей с уровнем сложности hard

In [30]:
purchase_df_hard = purchase_df[purchase_df["user_id"].isin(selected_level_hard_users)] # датафрейм с оплатой

In [31]:
purchase_df_hard['user_id'].nunique() #кол-во пользователей с оплатой  и уровнем easy

442

In [32]:
percent_of_purchase_hard = purchase_df_hard["user_id"].nunique() / len(
    selected_level_hard_users
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности hard): {:.2%}".format(
        percent_of_purchase_hard
    )
)

Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности hard): 35.39%


In [33]:
round(purchase_df_hard['amount'].mean(),2) # средний размер оплаты

111.6

- Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности easy): 7.72% 
- Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности medium): 20.86%
- Процент пользователей, которые оплатили тренировки (от числа пользователей с уровнем сложности hard): 35.39%

Как можно заметить процент процент пользователей, которые оплатили тренировки растёт с повышение уровня.
Так процент пользователей, которые оплатили тренировки с выбранным уровнем сложности hard в 4,6 раз выше чем у пользователей с уровнем easy

### Расчет временного промежутка между выбором уровня сложности и оплатой у групп пользователей с разным уровнем сложности

In [34]:
purchase_first_df = total_events_df[total_events_df["event_type"] == "purchase"]  # Создаем датафрейм, содержащий только первую покупку

In [35]:
purchase_first_df = purchase_first_df[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)            # Оставляем только нужные для вычислений столбцы

#### Уровень сложности easy

In [36]:
selected_level_easy_df_1 = selected_level_easy_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"})                # Оставляем только нужные для вычислений столбцы


In [37]:
selected_level_1 = selected_level_easy_df_1.merge(purchase_first_df, on="user_id", how="inner")

In [38]:
selected_level_1["timedelta"] = (
    selected_level_1["purchase_time"] - selected_level_1["level_choice_time"]
)
selected_level_1.head()               # Считаем временную разницу

,user_id,level_choice_time,purchase_time,timedelta
0,27884,2018-01-04 16:18:39,2018-01-08 19:37:34,4 days 03:18:55
1,28090,2018-01-09 21:34:23,2018-01-15 23:42:55,6 days 02:08:32
2,28182,2018-01-11 18:44:45,2018-01-12 02:46:01,0 days 08:01:16
3,28207,2018-01-11 21:10:51,2018-01-12 21:00:24,0 days 23:49:33
4,28254,2018-01-12 16:48:24,2018-01-19 22:08:40,7 days 05:20:16


In [39]:
selected_level_1["timedelta"].describe()

count                          189
mean     3 days 14:58:52.941798941
std      2 days 07:06:35.644097504
min                0 days 00:49:20
25%                1 days 17:18:56
50%                3 days 06:03:50
75%                5 days 06:58:18
max               10 days 18:35:09
Name: timedelta, dtype: object

#### Уровень сложности medium

In [40]:
selected_level_medium_df_1 = selected_level_medium_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"})                # Оставляем только нужные для вычислений столбцы


In [41]:
selected_level_2 = selected_level_medium_df_1.merge(purchase_first_df, on="user_id", how="inner")

In [42]:
selected_level_2["timedelta"] = (
    selected_level_2["purchase_time"] - selected_level_2["level_choice_time"]
)
selected_level_2.head()               # Считаем временную разницу

,user_id,level_choice_time,purchase_time,timedelta
0,27973,2018-01-07 05:29:30,2018-01-13 21:50:00,6 days 16:20:30
1,27981,2018-01-07 10:46:14,2018-01-07 23:20:25,0 days 12:34:11
2,28010,2018-01-08 00:00:52,2018-01-10 05:32:47,2 days 05:31:55
3,28020,2018-01-08 14:47:35,2018-01-11 21:43:03,3 days 06:55:28
4,28033,2018-01-08 17:06:39,2018-01-16 05:08:41,7 days 12:02:02


In [43]:
selected_level_2["timedelta"].describe()

count                          969
mean     3 days 23:14:13.165118679
std      2 days 06:18:57.618467109
min                0 days 04:18:12
25%                2 days 01:20:07
50%                3 days 19:53:19
75%                5 days 16:07:19
max               10 days 13:51:01
Name: timedelta, dtype: object

#### Уровень сложности hard

In [44]:
selected_level_hard_df_1 = selected_level_hard_df[["user_id", "start_time"]].rename(
    columns={"start_time": "level_choice_time"})                # Оставляем только нужные для вычислений столбцы


In [45]:
selected_level_3 = selected_level_hard_df_1.merge(purchase_first_df, on="user_id", how="inner")


In [46]:
selected_level_3["timedelta"] = (
    selected_level_3["purchase_time"] - selected_level_3["level_choice_time"]
)
selected_level_3.head()               # Считаем временную разницу

,user_id,level_choice_time,purchase_time,timedelta
0,27845,2018-01-02 06:19:18,2018-01-03 18:53:43,1 days 12:34:25
1,27865,2018-01-04 05:56:32,2018-01-04 14:46:10,0 days 08:49:38
2,27910,2018-01-05 11:59:50,2018-01-07 12:11:34,2 days 00:11:44
3,27911,2018-01-05 17:39:02,2018-01-07 08:19:12,1 days 14:40:10
4,27940,2018-01-06 00:32:47,2018-01-07 13:16:41,1 days 12:43:54


In [47]:
selected_level_3["timedelta"].describe()

count                          442
mean     3 days 07:20:41.420814479
std      1 days 21:43:52.953292605
min                0 days 03:26:45
25%         1 days 14:57:23.500000
50%         3 days 03:13:57.500000
75%         4 days 19:16:00.250000
max                8 days 01:18:13
Name: timedelta, dtype: object

Среднее время между выбором уровня сложности и покупкой:
- easy - 3 days 14:58:52;
- medium - 3 days 23:14:13;
- hard - 3 days 07:20:41.

Как можнм заметить среднее время между выбором уровня сложности и первой покупкой не сильно отличается по группам. Минимальное время между этими двумя событиями у группы пользователей с выбором уровня сложности hard. Следует также отметить, что у данных пользователей наблюдается минимальное значение максимального временного промежутка - разница с другими двумя уровнями сложностей составляет более двух с половиной суток.

### Расчет временного промежутка между регистрацией и оплатой у групп пользователей с разным уровнем сложности

In [48]:
registration_df = total_events_df[total_events_df['event_type'] == 'registration'] # выделяем отдельный датафрейм registration_df, содержащий только события с event_type = registration. 
                                                                                   # Этот датафрейм будет вспомогательным для определения времени между регистрацией и началом обучения

In [49]:
registration_df = registration_df[["user_id", "start_time"]].rename(
    columns={"start_time": "registration_time"}
) # Оставляем только нужные для вычислений столбцы

#### Уровень сложности easy

In [50]:
purchase_easy = purchase_df_easy[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)            # Оставляем только нужные для вычислений столбцы

In [51]:
purchase_reg_easy = purchase_easy.merge(registration_df, on="user_id", how="inner")


In [52]:
purchase_reg_easy["timedelta"] = (
    purchase_reg_easy["purchase_time"] - purchase_reg_easy["registration_time"]
)
purchase_reg_easy.head()               # Считаем временную разницу

,user_id,purchase_time,registration_time,timedelta
0,27884,2018-01-08 19:37:34,2018-01-04 11:50:43,4 days 07:46:51
1,28182,2018-01-12 02:46:01,2018-01-11 10:12:20,0 days 16:33:41
2,28207,2018-01-12 21:00:24,2018-01-11 16:27:37,1 days 04:32:47
3,28090,2018-01-15 23:42:55,2018-01-09 19:31:24,6 days 04:11:31
4,28378,2018-01-18 02:11:41,2018-01-15 14:55:57,2 days 11:15:44


In [53]:
purchase_reg_easy["timedelta"].describe()

count                          189
mean     3 days 22:10:23.211640211
std      2 days 07:14:41.062010764
min                0 days 04:36:58
25%                2 days 01:12:12
50%                3 days 11:00:23
75%                5 days 10:24:59
max               11 days 00:35:04
Name: timedelta, dtype: object

#### Уровень сложности medium

In [54]:
purchase_medium = purchase_df_medium[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)            # Оставляем только нужные для вычислений столбцы

In [55]:
purchase_reg_medium = purchase_medium.merge(registration_df, on="user_id", how="inner")


In [56]:
purchase_reg_medium["timedelta"] = (
    purchase_reg_medium["purchase_time"] - purchase_reg_medium["registration_time"]
)
purchase_reg_medium.head()               # Считаем временную разницу

,user_id,purchase_time,registration_time,timedelta
0,27981,2018-01-07 23:20:25,2018-01-07 08:09:09,0 days 15:11:16
1,28026,2018-01-09 20:51:27,2018-01-08 13:46:31,1 days 07:04:56
2,28010,2018-01-10 05:32:47,2018-01-07 22:19:23,2 days 07:13:24
3,28094,2018-01-10 13:44:50,2018-01-09 21:25:14,0 days 16:19:36
4,28035,2018-01-10 13:59:12,2018-01-08 17:49:55,1 days 20:09:17


In [57]:
purchase_reg_medium["timedelta"].describe()

count                          969
mean     4 days 06:12:06.576883384
std      2 days 06:25:57.480868026
min                0 days 08:39:24
25%                2 days 08:46:51
50%                4 days 03:35:26
75%                5 days 23:51:27
max               10 days 20:34:02
Name: timedelta, dtype: object

#### Уровень сложности hard

In [58]:
purchase_hard = purchase_df_hard[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)            # Оставляем только нужные для вычислений столбцы

In [59]:
purchase_reg_hard = purchase_hard.merge(registration_df, on="user_id", how="inner")

In [60]:
purchase_reg_hard["timedelta"] = (
    purchase_reg_hard["purchase_time"] - purchase_reg_hard["registration_time"]
)
purchase_reg_hard.head()               # Считаем временную разницу

,user_id,purchase_time,registration_time,timedelta
0,27845,2018-01-03 18:53:43,2018-01-02 01:35:56,1 days 17:17:47
1,27865,2018-01-04 14:46:10,2018-01-03 11:14:57,1 days 03:31:13
2,27911,2018-01-07 08:19:12,2018-01-05 10:48:24,1 days 21:30:48
3,27910,2018-01-07 12:11:34,2018-01-05 10:45:33,2 days 01:26:01
4,27940,2018-01-07 13:16:41,2018-01-05 23:41:24,1 days 13:35:17


In [61]:
purchase_reg_hard["timedelta"].describe()

count                          442
mean     3 days 14:55:19.257918552
std      1 days 22:22:52.441896774
min                0 days 09:41:39
25%         1 days 23:36:25.500000
50%         3 days 10:10:04.500000
75%         5 days 03:30:07.750000
max                8 days 14:21:29
Name: timedelta, dtype: object

Среднее время между регистрацией и оплатой по уровням сложности:
- easy - 3 days 22:10:23;
- medium - 4 days 06:12:06;
- hard - 3 days 14:55:19.

Как и в предыдущих данных среднее время между регистрацией и первой покупкой минимально у группы пользователей с выбором уровня сложности hard. Самое большре значение у группы пользователей с уровнем сложности medium. Следует также отметить, что у группы пользователей с выбором уровня сложности hard также наблюдается разница с другими двумя уровнями сложностей более двух с половиной суток в значении максимального временного промежутка.

Из приведённых данных можно сделать вывод, что группы пользователей с высоким уровнем сложности более расположены к покупкам